In [ ]:
from qiskit import *
from qiskit_aer import AerSimulator
import numpy as np
from copy import deepcopy

In [55]:
def function_tel(n=0):
    Alice = QuantumRegister(1,'Alice')
    Bob = QuantumRegister(1,'Bob')
    Secret = QuantumRegister(1,'Secret')
    classical = ClassicalRegister(2, 'c')

    circuit = QuantumCircuit(Secret,Alice,Bob,classical)

    # Se n>0, adiciona registrador extra "ext" com n qubits
    if n > 0:
        ext = QuantumRegister(n, 'ext')
        circuit = QuantumCircuit(Secret, Alice, Bob, ext, classical)
    else:
        circuit = QuantumCircuit(Secret, Alice, Bob, classical)
    
    #EPR
    circuit.h(Alice)
    circuit.cx(Alice,Bob)
    circuit.barrier()

    # swaps
    if n > 0:
        circuit.swap(Bob[0], ext[0])
        for i in range(1, n):
            circuit.swap(ext[i - 1], ext[i])
        final_target = ext[n - 1]
    else:
        final_target = Bob[0]


    #Passa a informação
    circuit.barrier()
    circuit.h(Secret)
    circuit.barrier()

    #Misturas de Estados
    circuit.cx(Secret,Alice)
    circuit.h(Secret)
    circuit.barrier()

    #Medições
    circuit.measure(Alice, classical[1])
    circuit.measure(Secret, classical[0])
    circuit.barrier()

    # Correções condicionais usando if_test()
    with circuit.if_test((classical, 1)):
        circuit.x(final_target)

    with circuit.if_test((classical, 2)):
        circuit.z(final_target)

    creg = ClassicalRegister(1, 'c_tom')
    circuit.add_register(creg)
    circuit.measure(2,creg[0])

    return circuit

In [ ]:
import numpy as np
from qiskit import *
from qiskit_aer import AerSimulator

pauli_list = [
    np.eye(2),
    np.array([[0.0, 1.0], [1.0, 0.0]]),
    np.array([[0, -1.0j], [1.0j, 0.0]]),
    np.array([[1.0, 0.0], [0.0, -1.0]]),
]

bases = {
    'I': pauli_list[0],
    'X': pauli_list[1],
    'Y': pauli_list[2],
    'Z': pauli_list[3],
}

backend = AerSimulator()

def measurement_circuit(base,input_circuit, target):
    circuit = deepcopy(input_circuit)


    if base == 'X':
        circuit.h(target)
    elif base == 'Y':
        circuit.sdg(target)
        circuit.h(target)

    return circuit

def tomography(my_circuit, target,shots):
    expectation_values = {}
    all_counts = {}

    for base in ['X','Y','Z']:
        circ       = measurement_circuit(base, my_circuit, target)
        transpiled = transpile(circ, backend)
        result     = backend.run(transpiled, shots=shots).result()
        counts     = result.get_counts()

        # Agora filtramos o primeiro bit da string (bits[0]), que corresponde a c_tom[0]
        p0 = sum(cnt for bits, cnt in counts.items() if bits[0]=='0')
        p1 = sum(cnt for bits, cnt in counts.items() if bits[0]=='1')
        total = p0 + p1 or 1

        expectation_values[base] = (p0 - p1) / total
        all_counts[base] = counts

    rho = 0.5 * (
        pauli_list[0]
        + expectation_values['X'] * pauli_list[1]
        + expectation_values['Y'] * pauli_list[2]
        + expectation_values['Z'] * pauli_list[3]
    )

    return rho, expectation_values



In [ ]:
def function_tel_tomography(n=0):
    Alice = QuantumRegister(1, 'Alice')
    Bob = QuantumRegister(1, 'Bob')
    Secret = QuantumRegister(1, 'Secret')
    classical = ClassicalRegister(2, 'c')
    c_tom = ClassicalRegister(1, 'c_tom')

    if n > 0:
        ext = QuantumRegister(n, 'ext')
        circuit = QuantumCircuit(Secret, Alice, Bob, ext, classical, c_tom)
    else:
        circuit = QuantumCircuit(Secret, Alice, Bob, classical, c_tom)

    #EPR
    circuit.h(Alice)
    circuit.cx(Alice, Bob)
    circuit.barrier()

    # swaps
    if n > 0:
        circuit.swap(Bob[0], ext[0])
        for i in range(1, n):
            circuit.swap(ext[i - 1], ext[i])
        final_target = ext[n - 1]
    else:
        final_target = Bob[0]

    #Passa a informação
    circuit.barrier()
    circuit.h(Secret)
    circuit.barrier()

    #Misturas de Estados
    circuit.cx(Secret, Alice)
    circuit.h(Secret)
    circuit.barrier()

    #Medições
    circuit.measure(Alice, classical[1])
    circuit.measure(Secret, classical[0])
    circuit.barrier()

    # Correções condicionais
    with circuit.if_test((classical, 1)):
        circuit.x(final_target)

    with circuit.if_test((classical, 2)):
        circuit.z(final_target)

    # Medida para tomografia
    pauli_list = [
    np.eye(2),
    np.array([[0.0, 1.0], [1.0, 0.0]]),
    np.array([[0, -1.0j], [1.0j, 0.0]]),
    np.array([[1.0, 0.0], [0.0, -1.0]]),
]

bases = {
    'I': pauli_list[0],
    'X': pauli_list[1],
    'Y': pauli_list[2],
    'Z': pauli_list[3],
}

backend = AerSimulator()

def measurement_circuit(base,input_circuit, target):
    circuit = deepcopy(input_circuit)


    if base == 'X':
        circuit.h(target)
    elif base == 'Y':
        circuit.sdg(target)
        circuit.h(target)

    return circuit

def tomography(my_circuit, target,shots):
    expectation_values = {}
    all_counts = {}

    for base in ['X','Y','Z']:
        circ       = measurement_circuit(base, my_circuit, target)
        transpiled = transpile(circ, backend)
        result     = backend.run(transpiled, shots=shots).result()
        counts     = result.get_counts()

        # Agora filtramos o primeiro bit da string (bits[0]), que corresponde a c_tom[0]
        p0 = sum(cnt for bits, cnt in counts.items() if bits[0]=='0')
        p1 = sum(cnt for bits, cnt in counts.items() if bits[0]=='1')
        total = p0 + p1 or 1

        expectation_values[base] = (p0 - p1) / total
        all_counts[base] = counts

    rho = 0.5 * (
        pauli_list[0]
        + expectation_values['X'] * pauli_list[1]
        + expectation_values['Y'] * pauli_list[2]
        + expectation_values['Z'] * pauli_list[3]
    )

    return rho, expectation_values 
